Simply Trying on Italy(Greedy algorithm)

In [1]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [2]:
#Read cities data and distance matrix

CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


In [3]:
#define cost
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

In [4]:
# Greedy algorithm
def greedy_tsp():
    best_path = None
    min_cost = float('inf')

    # Iterate over all starting cities
    for start_city in range(len(CITIES)):
        visited = np.full(len(CITIES), False)
        dist = DIST_MATRIX.copy()
        city = start_city
        visited[city] = True
        tsp = [city]

        logging.info(f"Starting from {CITIES.at[start_city, 'name']}")

        # Choose the closest city that hasn't been visited at each step
        while not np.all(visited):
            dist[:, city] = np.inf  # Mark the current city as visited
            closest = np.argmin(dist[city])
            visited[closest] = True
            tsp.append(closest)
            logging.debug(
                f"Visited {CITIES.at[city, 'name']} -> Closest city: {CITIES.at[closest, 'name']} "
                f"Distance: {DIST_MATRIX[city, closest]:.2f} km"
            )
            city = closest

        # Close the loop 
        tsp.append(start_city)
        path_cost = tsp_cost(tsp)

        # Check if it's the best solution
        if path_cost < min_cost:
            min_cost = path_cost
            best_path = tsp
            logging.info(f"New best path starting from {CITIES.at[start_city, 'name']} "
                         f"with distance {min_cost:.2f} km")

        # Output the complete path for the current starting city
        logging.info(f"Path for starting city {CITIES.at[start_city, 'name']}: "
                     f"{' -> '.join([CITIES.at[i, 'name'] for i in tsp])}, "
                     f"Total distance: {path_cost:.2f} km")

    return best_path, min_cost



In [5]:
best_path, min_cost = greedy_tsp()
logging.info(f"best route: {[CITIES.at[i, 'name'] for i in best_path]}")
logging.info(f"shortest total distance: {min_cost:.2f} km")

INFO:root:Starting from Ancona
DEBUG:root:Visited Ancona -> Closest city: Rimini Distance: 90.60 km
DEBUG:root:Visited Rimini -> Closest city: Forlì Distance: 46.72 km
DEBUG:root:Visited Forlì -> Closest city: Ravenna Distance: 26.46 km
DEBUG:root:Visited Ravenna -> Closest city: Ferrara Distance: 66.67 km
DEBUG:root:Visited Ferrara -> Closest city: Bologna Distance: 43.43 km
DEBUG:root:Visited Bologna -> Closest city: Modena Distance: 37.29 km
DEBUG:root:Visited Modena -> Closest city: Reggio nell'Emilia Distance: 23.94 km
DEBUG:root:Visited Reggio nell'Emilia -> Closest city: Parma Distance: 26.94 km
DEBUG:root:Visited Parma -> Closest city: Piacenza Distance: 57.65 km
DEBUG:root:Visited Piacenza -> Closest city: Milan Distance: 60.65 km
DEBUG:root:Visited Milan -> Closest city: Monza Distance: 14.51 km
DEBUG:root:Visited Monza -> Closest city: Bergamo Distance: 33.92 km
DEBUG:root:Visited Bergamo -> Closest city: Brescia Distance: 46.02 km
DEBUG:root:Visited Brescia -> Closest city: